In [ ]:
!pip install bs4==0.0.1

     |████████████████████████████████| 115 kB 14.4 MB/s 
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=45d465ac4630f2079808c9ffbd225f1ee37ac55f72431ba63708a7cd411a7ef6
  Stored in directory: /home/jovyan/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [ ]:
from bs4 import BeautifulSoup  # 对html格式的文件, 处理
import requests
import pandas as pd
from time import sleep

In [ ]:
def get_info(res, get_content=True):
    '''
    获取指定的内容(题目和评论)或问题id
    '''
    if res.status_code != 200:
        print("ip被封!, 等候10s")
        sleep(10)
        if get_content:
            return 0
        else:
            return []
    json = res.json()
    contents_dict = json["data"]
    if get_content:
        contents_list = []
        for item in contents_dict:
            content = ""
            title_name = item["question"]["title"]
            # voteup_count = item["voteup_count"]
            html = item["content"]
            bf = BeautifulSoup(html)
            for p in bf.stripped_strings:
                content += p
            contents_list.append((title_name, content))
        return contents_list
    else:
        question_id_list = []
        for item in contents_dict:
            content = ""
            question_id = str(item["id"])
            question_id_list.append(question_id)
        return question_id_list

In [ ]:
def grap_web(question_id):
    '''
    用于获取该问题号对应的所有评论
    '''
    # 爬取网站, num控制每次爬取评论数目
    num = 5
    offset = 0 # 起始值
    i = 0
    ## 准备爬取的网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/answers?include=data%5B*%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B*%5D.mark_infos%5B*%5D.url%3Bdata%5B*%5D.author.follower_count%2Cbadge%5B*%5D.topics%3Bsettings.table_of_content.enabled%3B&limit={}&offset={}&sort_by=default&platform=desktop"
    print("\n抓取知乎问题号{}:".format(question_id))
    while True:
        url = base_url.format(num, offset + i*num)
        ## 准备https头部
        headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
        # ## 睡眠
        sleep(0.75)
        ## 开始访问
        res = requests.get(url, headers=headers)
        ## 处理json数据, 获取回答和题目
        contents_list = get_info(res)
        ## 对于IP被封或请求失败时的处理
        if contents_list==0:
            print("再次申请...")
            continue
        ## 存储数据到csv中
        dataset = pd.DataFrame(contents_list, columns=["title", "comment"])
        if i==0:
            dataset.to_csv("data.csv", mode="a", header=False, index=False, sep="\t")
        else:
            dataset.to_csv("data.csv", mode="a", header=False, index=False, sep="\t")
        ## 判断截至条件
        if len(contents_list) < num:
            if len(contents_list) != 0:
                print("  已抓取{}条评论!".format(offset + i*num + len(contents_list)))
            print("问题号:" + question_id +"抓取完成!")
            break
        i += 1
        print("  已抓取{}条评论!".format(offset + i*num))

In [ ]:
def to_more_grap(question_id):
    '''
    用于获取相关问题的问题id
    '''
    # 准备网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/similar-questions?include=data%5B*%5D.answer_count%2Cauthor%2Cfollower_count&limit=5"
    
    ## 准备https头部
    headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    # ## 睡眠
    sleep(0.75)
    ## 开始访问
    res = requests.get(base_url, headers=headers)
    ## 处理json数据, 获取更多问题号
    contents_list = get_info(res, get_content=False)
    return contents_list

In [ ]:
def set_grap(question_id, question_num=1):
    '''
    集成爬虫
    '''
    # 爬取网站
    grap_web(question_id)
    question_num -= 1

    question_id = [question_id]
    i = 1
    while question_num:
        temp_i = i
        while len(question_id) == i:
            question_id += to_more_grap(question_id[temp_i-1])
            question_id = sorted(set(question_id),key=question_id.index)
            temp_i -= 1
            if temp_i == -1:
                print("因问题一致, 故抓取结束!")
                return
        grap_web(question_id[i])
        i += 1
        question_num -= 1
    print("抓取结束!")

In [ ]:
def main():
    # 默认id :339505719, 48510028, 20399824, 365863994, 20066579
    print("输入抓取知乎对应问题的号码:")
    question_id = input()
    print("输入成功!")
    set_grap(question_id, question_num=100)

if __name__=="__main__":
    main()

输入抓取知乎对应问题的号码:
输入成功!

抓取知乎问题号431580652:
  已抓取5条评论!
  已抓取9条评论!
问题号:431580652抓取完成!

抓取知乎问题号60521020:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取25条评论!
  已抓取30条评论!
  已抓取35条评论!
  已抓取40条评论!
  已抓取45条评论!
  已抓取50条评论!
  已抓取55条评论!
  已抓取60条评论!
  已抓取65条评论!
  已抓取70条评论!
  已抓取75条评论!
  已抓取80条评论!
  已抓取85条评论!
  已抓取90条评论!
  已抓取95条评论!
  已抓取100条评论!
  已抓取105条评论!
  已抓取110条评论!
  已抓取115条评论!
  已抓取120条评论!
  已抓取125条评论!
  已抓取130条评论!
  已抓取135条评论!
  已抓取140条评论!
  已抓取145条评论!
  已抓取150条评论!
  已抓取155条评论!
  已抓取160条评论!
  已抓取165条评论!
  已抓取170条评论!
  已抓取175条评论!
  已抓取180条评论!
  已抓取185条评论!
  已抓取190条评论!
  已抓取195条评论!
  已抓取200条评论!
  已抓取205条评论!
  已抓取210条评论!
  已抓取215条评论!
  已抓取220条评论!
  已抓取225条评论!
  已抓取230条评论!
  已抓取235条评论!
  已抓取240条评论!
  已抓取245条评论!
  已抓取250条评论!
  已抓取255条评论!
  已抓取260条评论!
  已抓取265条评论!
  已抓取270条评论!
  已抓取275条评论!
  已抓取280条评论!
  已抓取285条评论!
  已抓取290条评论!
  已抓取295条评论!
  已抓取300条评论!
  已抓取305条评论!
  已抓取310条评论!
  已抓取315条评论!
  已抓取320条评论!
  已抓取325条评论!
  已抓取330条评论!
  已抓取335条评论!
  已抓取340条评论!
  已抓取345条评论!
  已抓取350条评论!
  已抓取355条评论

**观察数据**

In [ ]:
import pandas as pd
dataset = pd.read_csv("data.csv", sep="\t", names=["title", "content"])

In [ ]:
dataset.head()

,title,content
0,特朗普在 G20 峰会发言称，《巴黎协定》旨在「杀死美国经济」，不是为了拯救环境，对此你怎么看？,他在给拜登挖坑，如果拜登加入巴黎协定、tpp，就等于背叛美国工人，他在为未来的斗争铺路，他们...
1,特朗普在 G20 峰会发言称，《巴黎协定》旨在「杀死美国经济」，不是为了拯救环境，对此你怎么看？,众所周知，美国一直是地球上人均碳排放最高的国家。而当中国制造最早做低端低附加值的产品时，美国...
2,特朗普在 G20 峰会发言称，《巴黎协定》旨在「杀死美国经济」，不是为了拯救环境，对此你怎么看？,我就是很好奇。如果为了环保，月薪6000降为3000（个人月薪，非平均，意味着生活质量减半）...
3,特朗普在 G20 峰会发言称，《巴黎协定》旨在「杀死美国经济」，不是为了拯救环境，对此你怎么看？,再次印证一件事:每届美国政府存在的目的，就是推翻上届政府的一些法规政策，同时立下新的法规政策...
4,特朗普在 G20 峰会发言称，《巴黎协定》旨在「杀死美国经济」，不是为了拯救环境，对此你怎么看？,可怜我川宝，春蚕到死丝方尽，蜡炬成灰泪始干。虐川一时爽，追川火葬场。美国人不感谢川宝，因为他...


In [ ]:
dataset["title"].value_counts()

如何评价特朗普决定退出《巴黎协定》？                 356
全球气候变化被高估还是低估了？                    179
美国说疫情过后要和中国算账，美国的疫情锅真的能甩到中国头上吗？    166
如何看待美媒预计川普宣布美国将退出《巴黎协定》？           133
如何看待唐纳德.特朗普宣布美国退出《巴黎协定》？            84
                                  ... 
谁可以减缓温室效应?                           5
减缓全球变暖快速增长的最有效的正确方法是什么？              5
人类对地球变暖有什么应对措施吗？                     4
看电视会导致全球变暖么？                         4
全球变暖会不会导致有一天会世界末日？                   4
Name: title, Length: 100, dtype: int64